In [19]:
from sklearn.preprocessing import OneHotEncoder

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score,precision_score,recall_score,f1_score, roc_auc_score

In [20]:
df = pd.read_csv('../data/crawler/unified-events-statistics.csv')
df = df.drop(['golId','game','event4','event5','event6','event7','event8','event9','event10','event11','event12','event13','event14','event15','event16','event17','event18','event19','event20','event21','event22','event23','event24','event25','event26','event27','event28','event29','event30','event31','event32','event33','event34','event35','event36','event37','event38','event39','event40','event41','event42','event43','event44','event45'],axis=1)
y = df.result
x = df.drop(['result'],axis=1)
df.columns

Index(['event1', 'event2', 'event3', 'blueTopGP', 'blueTopWR', 'blueTopKDA',
       'blueJungleGP', 'blueJungleWR', 'blueJungleKDA', 'blueMidGP',
       'blueMidWR', 'blueMidKDA', 'blueADCGP', 'blueADCWR', 'blueADCKDA',
       'blueSupportGP', 'blueSupportWR', 'blueSupportKDA', 'redTopGP',
       'redTopWR', 'redTopKDA', 'redJungleGP', 'redJungleWR', 'redJungleKDA',
       'redMidGP', 'redMidWR', 'redMidKDA', 'redAdcGP', 'redAdcWR',
       'redAdcKDA', 'redSupportGP', 'redSupportWR', 'redSupportKDA', 'result'],
      dtype='object')

In [21]:
#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')

#perform one-hot encoding on 'team' column 
encoder_df = pd.DataFrame(encoder.fit_transform(df[['event1', 'event2', 'event3']]).toarray())
encoder_df.columns = encoder.get_feature_names()
#merge one-hot encoded columns back with original DataFrame
transformed_df = df.join(encoder_df)
transformed_df = transformed_df.drop(['event1', 'event2', 'event3'],axis=1)
transformed_df.head()

,blueTopGP,blueTopWR,blueTopKDA,blueJungleGP,blueJungleWR,blueJungleKDA,blueMidGP,blueMidWR,blueMidKDA,blueADCGP,...,x2_BLUE: first_tower_bot,x2_BLUE: first_tower_mid,x2_BLUE: first_tower_top,x2_BLUE: herald,x2_RED: dragon,x2_RED: first_blood,x2_RED: first_tower_bot,x2_RED: first_tower_mid,x2_RED: first_tower_top,x2_RED: herald
0,1,1.00,2.0,1,1.0,2.3,1,1.00,2.7,6,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5,0.60,5.6,2,0.0,1.1,9,0.33,1.7,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0.00,0.0,1,1.0,3.3,0,0.00,0.0,2,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,23,0.61,3.6,5,0.2,1.6,8,0.63,4.2,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,3,1.00,9.8,2,1.0,7.0,1,0.00,2.0,7,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
corr = transformed_df.corr()

transformed_df[transformed_df.columns[1:]].corr()['result'][:-1].sort_values(ascending=False)

result                      1.000000
x0_BLUE: first_blood        0.192538
x1_BLUE: dragon             0.100601
blueJungleWR                0.100301
x2_BLUE: herald             0.090096
blueMidWR                   0.089401
x0_BLUE: dragon             0.088713
blueADCWR                   0.079716
blueJungleKDA               0.079660
x1_BLUE: herald             0.075754
blueSupportWR               0.071892
blueTopWR                   0.070487
x2_BLUE: first_tower_bot    0.063549
blueMidKDA                  0.063028
x2_BLUE: first_tower_top    0.061251
blueMidGP                   0.060392
blueADCKDA                  0.056527
blueSupportKDA              0.055146
x1_BLUE: first_blood        0.053311
x2_BLUE: first_tower_mid    0.052798
x2_BLUE: first_blood        0.046731
blueJungleGP                0.042915
blueSupportGP               0.041416
x2_BLUE: dragon             0.040805
blueTopKDA                  0.037603
x1_RED: first_tower_bot     0.020426
x0_BLUE: herald             0.020065
x

In [23]:
y = transformed_df['result'].copy()
X = transformed_df.drop(['result'],axis=1)

In [24]:
def preprocess_input(X,y):
    X = X.copy()
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42, stratify=y)
    return X_train,X_test,y_train,y_test
X_train,X_test,y_train,y_test = preprocess_input(X,y)

In [25]:
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier

models = {
    'Logistic Regression': LogisticRegression(max_iter=100000),
    'Support Vector Machine (Linear Kernel)': LinearSVC(max_iter=100000),
    'Support Vector Machine (RBF Kernel)': SVC(max_iter=100000),
    'Decission Tree': DecisionTreeClassifier(),
    'Adaboost': AdaBoostClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting Classifier': GradientBoostingClassifier()
}


for name, model in models.items():
  model.fit(X_train,y_train)
  print(name + ' trained')

Logistic Regression trained


c:\Users\linco\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Support Vector Machine (Linear Kernel) trained
Support Vector Machine (RBF Kernel) trained
Decission Tree trained
Adaboost trained
Random Forest trained
Gradient Boosting Classifier trained


In [26]:
scores_list = []

for name,model in models.items():    
    scores_list.append({
    'Model': name,
    'Balanced Accuracy': balanced_accuracy_score(y_test,model.predict(X_test)),
    'Precision':  precision_score(y_test,model.predict(X_test)),
    'Recall': recall_score(y_test,model.predict(X_test)),
    'F1-Score': f1_score(y_test,model.predict(X_test)),
    'ROC/AUC Score': roc_auc_score(y_test,model.predict(X_test))
    })
scores = pd.DataFrame(scores_list)

In [27]:
scores

,Model,Balanced Accuracy,Precision,Recall,F1-Score,ROC/AUC Score
0,Logistic Regression,0.652765,0.651911,0.690832,0.670807,0.652765
1,Support Vector Machine (Linear Kernel),0.653927,0.653846,0.688699,0.670820,0.653927
2,Support Vector Machine (RBF Kernel),0.610000,0.610236,0.660981,0.634596,0.610000
3,Decission Tree,0.581004,0.594595,0.562900,0.578313,0.581004
4,Adaboost,0.654401,0.660338,0.667377,0.663839,0.654401
5,Random Forest,0.669374,0.670782,0.695096,0.682723,0.669374
6,Gradient Boosting Classifier,0.657315,0.658487,0.686567,0.672234,0.657315
